In this notebook, we will show how to conduct co-simulation for an air-cooled data center with one CRAC unit and a chiller plant. In this case, the control variables are the the CRAC supply air flow rate and the supply air temperature setpoint.

In [ ]:
import numpy as np

from dctwin.interfaces.gym_envs import CoSimEnv
from dctwin.utils import config as env_config
from dctwin.utils import read_engine_config, setup_logging
from hooks import map_boundary_condition_fn
from google.protobuf import json_format

(1) Setup environment variables

In [ ]:
engine_config = "example.prototxt"
env_config.eplus.engine_config_file = engine_config

(2) Read configuration files

In [ ]:
config = read_engine_config(engine_config=engine_config)
setup_logging(config.logging_config, engine_config=engine_config)

(3) Build environment

In [ ]:
env_config_name = config.WhichOneof("EnvConfig")
env_params = json_format.MessageToDict(
    getattr(config, env_config_name).env_params,
    preserving_proto_field_name=True,
)
env = CoSimEnv(
    config=getattr(config, env_config_name),
    reward_fn=None,
    schedule_fn=None,
    map_boundary_condition_fn=map_boundary_condition_fn,
    **env_params,
)
env.reset()

(4) Specify the action dictionary

In [ ]:
action_dict = {
    "ACU1_setpoint": 20.0,
    "ACU1_flow_rate": 15.0,
    "chw_supply_sp": 10.0
}

Note: the action_dict should be in the format of: { "{uid}_setpoint": xxx, "{uid}_flow_rate": xxx,  ...(other actions for the chiller plant side (e.g., chilled water temperature, ....) to be sent to EnergyPlus) } The "uid" can be obtained by running the previous cell. Each CRAC should have a setpoint and a flow rate.

(5) Run energy simulation with fixed action to obtain energy consumption result Note: we highly recommend you to open the dry run mode to speed up simulation as we use the fixed action to run the simulation and we should not run CFD simulation again! Instead, we use the cached CFD result to pass to EnergyPlus


In [ ]:
act = np.array([
    action_dict["ACU1_setpoint"],
    action_dict["ACU1_flow_rate"],
    action_dict["chw_supply_sp"]
])
done = False
env_config.cfd.dry_run = True
env_config.cfd.mesh_dir = env_config.LOG_DIR.joinpath("base")
while not done:
    obs, rew, done, truncated, info = env.step(act)